In [73]:
import json
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import ComplementNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
import numpy as np
from nltk.stem import *
from tqdm import tqdm

In [81]:
# Opens the scraped file to read the data
with open('scps.json') as json_file:
    data = json.load(json_file)

# Checks that entries are complete, discards otherwise
clean_data = {id : data[id] for id in data if data[id]['obj_class'] in ['Safe', 'Euclid', 'Keter'] and data[id]['desc'] != '' and data[id]['spconproc'] != ''}
discarded = [id for id in list(data.keys()) if id not in list(clean_data.keys())]
print("Discarded " + str(len(discarded)) + " SCP(s) due to non-conformity (one or more sections missing in wiki / object class different from 3 original ones).")

#Creates subsections of the received data
ids = [str(id) for id in list(clean_data.keys())]
obj_classes = [entry['obj_class'] for entry in list(clean_data.values())]
descs = [entry['desc'] for entry in list(clean_data.values())]
spconprocs = [entry['spconproc'] for entry in list(clean_data.values())]

# Creates stemmed versions of the descriptions and the special containment procedures
stemmer = PorterStemmer()
stemmed_descs = []
stemmed_spconprocs = []

# Stemmed total data
stemmed_clean_data = clean_data
for scp in clean_data.keys():
    s_desc = ' '.join([stemmer.stem(word) for word in clean_data[scp]['desc']])
    s_spconproc = ' '.join([stemmer.stem(word) for word in clean_data[scp]['spconproc']])
    stemmed_clean_data[scp]['desc'] = s_desc
    stemmed_clean_data[scp]['spconproc'] = s_spconproc

# Stemmed Descriptions
for i in range(len(descs)):
    stemmed_desc = []
    for word in descs[i].split():
        stemmed_desc.append(stemmer.stem(word))
    stemmed_descs.append(' '.join(stemmed_desc))

# Stemmed Sp. Con. Procs
for i in range(len(spconprocs)):
    stemmed_spconproc = []
    for word in spconprocs[i].split():
        stemmed_spconproc.append(stemmer.stem(word))
    stemmed_spconprocs.append(' '.join(stemmed_spconproc))

Discarded 2124 SCP(s) due to non-conformity (one or more sections missing in wiki / object class different from 3 original ones).


In [82]:
# Casts object classes to numbers to more easily model the data
obj_class_dict = {
    "safe" : 0,
    "euclid" : 1,
    "keter" : 2
}

# Method to go from the old containment class to the int version, used later
def int_obj_class(cont_class):
    return obj_class_dict[cont_class.lower()]

In [83]:
# IMPORTANT: Run to set everything up for the testing.
# Please ignore redundant declarations, they were made for order's sake.

# Models
gnb = GaussianNB()
mnb = MultinomialNB()
cnb = ComplementNB()

# Datasets
descs = descs
spconprocs = spconprocs
stemmed_descs = stemmed_descs
stemmed_spconprocs = stemmed_spconprocs
both = []
for i in range(len(clean_data)):
    both.append(descs[i] + ' ' + spconprocs[i])

stemmed_both = []
for i in range(len(clean_data)):
    stemmed_both.append(stemmed_descs[i] + ' ' + stemmed_spconprocs[i])
y = np.array([int_obj_class(obj_class) for obj_class in obj_classes])

# Vectorizers
v_tfidf = TfidfVectorizer(stop_words='english')
v_count = CountVectorizer(stop_words='english')

# Training set selector
skf = StratifiedKFold(n_splits=5)

In [84]:
#Variables, see options above
model = cnb
x = both
vectorizer = v_count

# Training and testing, do not edit
scores = []
for train_index, test_index in tqdm(skf.split(x, y), total=5):
    x_train, x_test = np.array(x)[train_index], np.array(x)[test_index]
    y_train, y_test = np.array(y)[train_index], np.array(y)[test_index]
    x_train = vectorizer.fit_transform(x_train).toarray()
    x_test = vectorizer.transform(x_test).toarray()
    model.fit(x_train, y_train)
    scores.append(model.score(x_test, y_test))
    
print("Score: " + str(np.average(scores)))

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:40<00:00,  8.05s/it]

Score: 0.5517948717948717


In [78]:
safe_class_prob_sorted = model.feature_log_prob_[0, :].argsort()[::-1]
euclid_class_prob_sorted = model.feature_log_prob_[1, :].argsort()[::-1]
keter_class_prob_sorted = model.feature_log_prob_[2, :].argsort()[::-1]

print("Most informative tokens for Object Class Safe:")
print(np.take(vectorizer.get_feature_names(), safe_class_prob_sorted[:15]))
print("\nMost informative tokens for Object Class Euclid:")
print(np.take(vectorizer.get_feature_names(), euclid_class_prob_sorted[:15]))
print("\nMost informative tokens for Object Class Keter:")
print(np.take(vectorizer.get_feature_names(), keter_class_prob_sorted[:15]))

Most informative tokens for Object Class Safe:
['testing' 'personnel' 'kept' 'containment' 'storage' 'locker' 'site'
 'level' 'stored' 'standard' 'instances' 'room' 'security' 'access'
 'contained']

Most informative tokens for Object Class Euclid:
['personnel' 'containment' 'instances' 'foundation' 'site' 'chamber'
 'area' 'kept' 'cell' 'times' 'contained' 'security' 'standard' 'level'
 'testing']

Most informative tokens for Object Class Keter:
['containment' 'personnel' 'foundation' 'instances' 'area' 'event'
 'immediately' 'individuals' 'chamber' 'mtf' 'time' 'site' 'kept'
 'contained' 'force']


In [8]:
# Just to check how many SCPs there are that belong to each object class
np.unique(y, return_counts=True)

(array([0, 1, 2]), array([1814, 2093,  968], dtype=int64))

In [63]:
from sklearn.inspection import permutation_importance

imps = permutation_importance(model, x_test, y_test)
importances = imps.importances_mean
std = imps.importances_std
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(x_test.shape[1]):
    print("%d. %s (%f)" % (f + 1, vectorizer.get_feature_names()[indices[f]], importances[indices[f]]))

Feature ranking:
1. pages (0.038462)
2. scps (0.038462)
3. book (0.038462)
4. cm (0.038462)
5. target (0.038462)
6. accidentally (0.038462)
7. picture (0.038462)
8. opened (0.038462)
9. document (0.038462)
10. testing (0.038462)
11. good (0.038462)
12. determine (0.038462)
13. black (0.038462)
14. investigations (0.038462)
15. command (0.038462)
16. limits (0.038462)
17. containing (0.038462)
18. exact (0.030769)
19. approximately (0.023077)
20. wear (0.023077)
21. light (0.015385)
22. containment (0.015385)
23. cell (0.015385)
24. subject (0.015385)
25. potentially (0.015385)
26. guards (0.015385)
27. point (0.015385)
28. room (0.015385)
29. observation (0.015385)
30. meter (0.015385)
31. meters (0.015385)
32. monitored (0.015385)
33. area (0.015385)
34. level (0.015385)
35. properties (0.015385)
36. gravity (0.007692)
37. position (0.007692)
38. phase (0.007692)
39. physical (0.007692)
40. director (0.007692)
41. cameras (0.007692)
42. weaponry (0.007692)
43. male (0.007692)
44. mani

342. failure (0.000000)
343. filtration (0.000000)
344. fatally (0.000000)
345. fate (0.000000)
346. file (0.000000)
347. fell (0.000000)
348. female (0.000000)
349. filed (0.000000)
350. fence (0.000000)
351. fenced (0.000000)
352. fencedoff (0.000000)
353. fencing (0.000000)
354. fern (0.000000)
355. ferns (0.000000)
356. fertilized (0.000000)
357. files (0.000000)
358. fervently (0.000000)
359. figures (0.000000)
360. fiber (0.000000)
361. figure (0.000000)
362. fibers (0.000000)
363. fibroin (0.000000)
364. field (0.000000)
365. fiery (0.000000)
366. fifteenmeter (0.000000)
367. feet (0.000000)
368. feelings (0.000000)
369. filters (0.000000)
370. feature (0.000000)
371. father (0.000000)
372. fatigue (0.000000)
373. fatigues (0.000000)
374. faulty (0.000000)
375. fbi (0.000000)
376. fear (0.000000)
377. filmy (0.000000)
378. filmed (0.000000)
379. film (0.000000)
380. features (0.000000)
381. feeling (0.000000)
382. feces (0.000000)
383. fed (0.000000)
384. feds (0.000000)
385. fe

712. elephant (0.000000)
713. elephants (0.000000)
714. elevated (0.000000)
715. eliminated (0.000000)
716. elimination (0.000000)
717. elongated (0.000000)
718. elucidated (0.000000)
719. emaciated (0.000000)
720. emaciation (0.000000)
721. embedded (0.000000)
722. embedding (0.000000)
723. emblems (0.000000)
724. elementary (0.000000)
725. emerge (0.000000)
726. emerged (0.000000)
727. emergencies (0.000000)
728. emergency (0.000000)
729. emerging (0.000000)
730. eminent (0.000000)
731. emission (0.000000)
732. emissions (0.000000)
733. emit (0.000000)
734. emits (0.000000)
735. emitted (0.000000)
736. elements (0.000000)
737. element (0.000000)
738. doll (0.000000)
739. elapsed (0.000000)
740. effort (0.000000)
741. efforts (0.000000)
742. egg (0.000000)
743. eggs (0.000000)
744. eggs1javascript (0.000000)
745. egypt (0.000000)
746. egyptian (0.000000)
747. eighteen (0.000000)
748. eighty (0.000000)
749. ejected (0.000000)
750. elaborate (0.000000)
751. elastic (0.000000)
752. elect

1075. hypothesised (0.000000)
1076. i136a (0.000000)
1077. i136c (0.000000)
1078. ia (0.000000)
1079. ibm (0.000000)
1080. iceberg (0.000000)
1081. id (0.000000)
1082. identical (0.000000)
1083. identifiable (0.000000)
1084. identification (0.000000)
1085. identified (0.000000)
1086. identify (0.000000)
1087. identifying (0.000000)
1088. identities (0.000000)
1089. identity (0.000000)
1090. idols (0.000000)
1091. igniting (0.000000)
1092. impale (0.000000)
1093. impenetrable (0.000000)
1094. incisions (0.000000)
1095. improved (0.000000)
1096. improvements (0.000000)
1097. improvisational (0.000000)
1098. impulse (0.000000)
1099. impurities (0.000000)
1100. inability (0.000000)
1101. inaccessible (0.000000)
1102. inactive (0.000000)
1103. inadvertent (0.000000)
1104. inadvertently (0.000000)
1105. inappropriate (0.000000)
1106. inaugural (0.000000)
1107. incapability (0.000000)
1108. incapable (0.000000)
1109. incapacitate (0.000000)
1110. incapacitated (0.000000)
1111. incapacitation 

1435. greenhouse (0.000000)
1436. greenland (0.000000)
1437. greenwich (0.000000)
1438. grime (0.000000)
1439. grip (0.000000)
1440. gripped (0.000000)
1441. groan (0.000000)
1442. groove (0.000000)
1443. gross (0.000000)
1444. grasping (0.000000)
1445. graphing (0.000000)
1446. glowing (0.000000)
1447. granted (0.000000)
1448. glows (0.000000)
1449. goals (0.000000)
1450. goats (0.000000)
1451. goggles (0.000000)
1452. going (0.000000)
1453. gold (0.000000)
1454. gone (0.000000)
1455. got (0.000000)
1456. gotten (0.000000)
1457. government (0.000000)
1458. grab (0.000000)
1459. grabbed (0.000000)
1460. grabbing (0.000000)
1461. grade (0.000000)
1462. grader (0.000000)
1463. gradual (0.000000)
1464. gradually (0.000000)
1465. graduate (0.000000)
1466. grain (0.000000)
1467. grains (0.000000)
1468. grams (0.000000)
1469. grandeur (0.000000)
1470. granite (0.000000)
1471. gang (0.000000)
1472. gampo (0.000000)
1473. gamma6 (0.000000)
1474. forensic (0.000000)
1475. foresee (0.000000)
147

...

5418. ms (0.000000)
5419. mtf (0.000000)
5420. mtys (0.000000)
5421. mu4 (0.000000)
5422. mucosal (0.000000)
5423. mucous (0.000000)
5424. mucus (0.000000)
5425. mucuslike (0.000000)
5426. mug (0.000000)
5427. multiple (0.000000)
5428. mount (0.000000)
5429. mottled (0.000000)
5430. motorized (0.000000)
5431. motorcycle (0.000000)
5432. mon (0.000000)
5433. monastery (0.000000)
5434. money (0.000000)
5435. monitor (0.000000)
5436. monitoring (0.000000)
5437. monk (0.000000)
5438. monks (0.000000)
5439. monofilament (0.000000)
5440. month (0.000000)
5441. monthly (0.000000)
5442. months (0.000000)
5443. moon (0.000000)
5444. moonshine (0.000000)
5445. morning (0.000000)
5446. morphed (0.000000)
5447. mosquito (0.000000)
5448. mosquitoes (0.000000)
5449. mother (0.000000)
5450. motifs (0.000000)
5451. motile (0.000000)
5452. motion (0.000000)
5453. motionless (0.000000)
5454. motor (0.000000)
5455. multiplication (0.000000)
5456. multiplying (0.000000)
5457. multitool (0.000000)
5458. na

5756. local (0.000000)
5757. lobby (0.000000)
5758. loads (0.000000)
5759. linen (0.000000)
5760. lines (0.000000)
5761. lining (0.000000)
5762. linked (0.000000)
5763. linnell (0.000000)
5764. lions (0.000000)
5765. lips (0.000000)
5766. liquefaction (0.000000)
5767. liquefies (0.000000)
5768. liquid (0.000000)
5769. liquidation (0.000000)
5770. liquidlike (0.000000)
5771. liquids (0.000000)
5772. list (0.000000)
5773. listed (0.000000)
5774. lit (0.000000)
5775. little (0.000000)
5776. live (0.000000)
5777. liver (0.000000)
5778. liverworts (0.000000)
5779. living (0.000000)
5780. loaded (0.000000)
5781. loading (0.000000)
5782. looking (0.000000)
5783. lookout (0.000000)
5784. loop (0.000000)
5785. m3 (0.000000)
5786. machinery (0.000000)
5787. machines (0.000000)
5788. mad (0.000000)
5789. magazine (0.000000)
5790. magicsleightofhand (0.000000)
5791. magnet (0.000000)
5792. magnetic (0.000000)
5793. magnetron (0.000000)
5794. magnitude (0.000000)
5795. magnitudes (0.000000)
5796. m

6128. mediumterm (0.000000)
6129. megalomania (0.000000)
6130. melody (0.000000)
6131. melted (0.000000)
6132. melting (0.000000)
6133. member (0.000000)
6134. members (0.000000)
6135. membrane (0.000000)
6136. measures (0.000000)
6137. measured (0.000000)
6138. massively (0.000000)
6139. measure (0.000000)
6140. match (0.000000)
6141. matched (0.000000)
6142. matching (0.000000)
6143. material (0.000000)
6144. materialize (0.000000)
6145. materials (0.000000)
6146. mathematical (0.000000)
6147. mathematics (0.000000)
6148. matrix (0.000000)
6149. matter (0.000000)
6150. mattress (0.000000)
6151. maturation (0.000000)
6152. mature (0.000000)
6153. matured (0.000000)
6154. maximum (0.000000)
6155. mazes (0.000000)
6156. mcd (0.000000)
6157. meals (0.000000)
6158. mealtimes (0.000000)
6159. mean (0.000000)
6160. meaning (0.000000)
6161. means (0.000000)
6162. measurable (0.000000)
6163. park (0.000000)
6164. partial (0.000000)
6165. partially (0.000000)
6166. reflects (0.000000)
6167. re

6477. right (0.000000)
6478. righting (0.000000)
6479. rigid (0.000000)
6480. rigidly (0.000000)
6481. ring (0.000000)
6482. ringmasterhermanfullerhub (0.000000)
6483. rings (0.000000)
6484. rip (0.000000)
6485. ripe (0.000000)
6486. ripen (0.000000)
6487. ripens (0.000000)
6488. ripped (0.000000)
6489. ripples (0.000000)
6490. respiratory (0.000000)
6491. respirator (0.000000)
6492. remaining (0.000000)
6493. replacements (0.000000)
6494. repairs (0.000000)
6495. repeat (0.000000)
6496. repeated (0.000000)
6497. repeatedly (0.000000)
6498. repeating (0.000000)
6499. repellent (0.000000)
6500. repercussions (0.000000)
6501. repetitive (0.000000)
6502. replace (0.000000)
6503. replaced (0.000000)
6504. replacement (0.000000)
6505. replaces (0.000000)
6506. respiration (0.000000)
6507. replacing (0.000000)
6508. replicate (0.000000)
6509. replicating (0.000000)
6510. report (0.000000)
6511. reported (0.000000)
6512. reportedly (0.000000)
6513. reports (0.000000)
6514. reposition (0.00000

6842. proves (0.000000)
6843. provide (0.000000)
6844. proposals (0.000000)
6845. proposal (0.000000)
6846. proportional (0.000000)
6847. proportion (0.000000)
6848. progress (0.000000)
6849. progression (0.000000)
6850. prohibited (0.000000)
6851. project (0.000000)
6852. projection (0.000000)
6853. projects (0.000000)
6854. prolonged (0.000000)
6855. prolonging (0.000000)
6856. prometheus (0.000000)
6857. prominently (0.000000)
6858. promote (0.000000)
6859. promotes (0.000000)
6860. promptly (0.000000)
6861. prompts (0.000000)
6862. prone (0.000000)
6863. pronounced (0.000000)
6864. proofing (0.000000)
6865. propagation (0.000000)
6866. propel (0.000000)
6867. proper (0.000000)
6868. properly (0.000000)
6869. property (0.000000)
6870. prophecies (0.000000)
6871. provided (0.000000)
6872. provisional (0.000000)
6873. provocative (0.000000)
6874. purely (0.000000)
6875. purpose (0.000000)
6876. purposes (0.000000)
6877. pursued (0.000000)
6878. pushed (0.000000)
6879. pushing (0.00000